<a href="https://colab.research.google.com/github/AdiVM/Neuro240/blob/main/AM_Neuro240_FinalProject_Submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the code for my final assignment. Note that for ease and readability, all my previous code is available with its respective checkpoints here: https://github.com/AdiVM/Neuro240

Much of my code was written for these checkpoints, and rather than submit it all again as one ipynb, I have just uploaded the respective code to github.

In [ ]:
# All future runs can start here
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
import os

Mounted at /content/drive


In [ ]:
metadata_path = "/content/drive/MyDrive/NIH_ChestXRay_Data_Neuro240/Data_Entry_2017_v2020.csv"
image_folder = "/content/drive/MyDrive/NIH_ChestXRay_Data_Neuro240/images"

metadata = pd.read_csv(metadata_path)

print("Metdata loaded")

# Filtering the metadata to find images labeled either no finding or those containing the word mass
filtered_metadata = metadata[
    (metadata["Finding Labels"] == "No Finding") |
    (metadata["Finding Labels"].str.contains("Mass", na=False))
]

filtered_image_indexes = set(filtered_metadata["Image Index"])
filtered_metadata = filtered_metadata.head(50000)

matching_images = sorted(list(filtered_metadata["Image Index"]))

# Convert to stored list
matching_images = sorted(list(matching_images))

print(f"Total matching images found: {len(matching_images)}")

Metdata loaded
Total matching images found: 50000


In [ ]:
# Now to perform stratified shuffle split
from sklearn.model_selection import train_test_split
import pandas as pd

 # Check class distribution before splitting
print(filtered_metadata["Finding Labels"].value_counts())

# There are many small classes of mass, so need to group them all together before splitting
# Standardize labels: Convert anything containing "Mass" to just "Mass"
filtered_metadata["Finding Labels"] = filtered_metadata["Finding Labels"].apply(
    lambda x: "Mass" if "Mass" in x else x
)

# Verify new label counts
print(filtered_metadata["Finding Labels"].value_counts())


Finding Labels
No Finding                                                            45542
Mass                                                                   1708
Infiltration|Mass                                                       329
Mass|Nodule                                                             293
Effusion|Mass                                                           285
                                                                      ...  
Atelectasis|Cardiomegaly|Consolidation|Effusion|Mass|Nodule               1
Atelectasis|Consolidation|Effusion|Mass|Nodule|Pleural_Thickening         1
Cardiomegaly|Consolidation|Effusion|Mass|Nodule|Pleural_Thickening        1
Cardiomegaly|Consolidation|Effusion|Infiltration|Mass|Nodule              1
Edema|Fibrosis|Infiltration|Mass                                          1
Name: count, Length: 258, dtype: int64
Finding Labels
No Finding    45542
Mass           4458
Name: count, dtype: int64


In [ ]:
label_map = {"No Finding": 0, "Mass": 1}
filtered_metadata["Label"] = filtered_metadata["Finding Labels"].map(label_map)

In [ ]:
# Split the data while ensuring proportional distribution of classes
train_metadata, test_metadata = train_test_split(
    filtered_metadata,
    test_size=0.2,
    stratify=filtered_metadata["Label"],
    random_state=42
)

# Class distribution in train and test sets
print("Training set:")
print(train_metadata["Label"].value_counts())

print("Testing Set:")
print(test_metadata["Label"].value_counts())

Training set:
Label
0    36434
1     3566
Name: count, dtype: int64
Testing Set:
Label
0    9108
1     892
Name: count, dtype: int64


In [ ]:
print(f"Train metadata entries: {len(train_metadata)}")
print(f"Test metadata entries: {len(test_metadata)}")

Train metadata entries: 40000
Test metadata entries: 10000


In [ ]:
# Filtering metdata
# Convert "Image Index" column to a set for fast lookup
train_image_files = set(train_metadata["Image Index"])
test_image_files = set(test_metadata["Image Index"])

train_images = sorted(list(train_image_files))
test_images = sorted(list(test_image_files))

# Convert to sorted lists for consistency
train_images = sorted(list(train_images))
test_images = sorted(list(test_images))


print(f"Total train images found: {len(train_images)}")
print(f"Total test images found: {len(test_images)}")

# Print a few samples
print("Sample train images:", train_images[:10])
print("Sample test images:", test_images[:10])

Total train images found: 40000
Total test images found: 10000
Sample train images: ['00000002_000.png', '00000004_000.png', '00000005_000.png', '00000005_002.png', '00000005_003.png', '00000005_004.png', '00000005_005.png', '00000006_000.png', '00000007_000.png', '00000011_002.png']
Sample test images: ['00000005_001.png', '00000008_001.png', '00000011_001.png', '00000011_003.png', '00000013_000.png', '00000013_017.png', '00000013_029.png', '00000013_030.png', '00000018_000.png', '00000032_049.png']


In [ ]:
# I will use TensorFlow for model training
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import pandas as pd

In [ ]:
# Image preprocessing parameters
image_size = (224, 224)  # Resize images
batch_size = 32

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# Only rescale for testing
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Load train images from directory
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_metadata,
    directory=image_folder,
    x_col="Image Index",
    y_col="Label",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="raw"
)

# Load test images
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_metadata,
    directory=image_folder,
    x_col="Image Index",
    y_col="Label",
    target_size=image_size,
    batch_size=batch_size,
    class_mode="raw"
)

Found 39997 validated image filenames.


/usr/local/lib/python3.11/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 3 invalid image filename(s) in x_col="Image Index". These filename(s) will be ignored.
  warnings.warn(


Found 10000 validated image filenames.


In [ ]:
print("Train distribution:")
print(train_metadata["Label"].value_counts())

print("\nTest distribution:")
print(test_metadata["Label"].value_counts())

Train distribution:
Label
0    36434
1     3566
Name: count, dtype: int64

Test distribution:
Label
0    9108
1     892
Name: count, dtype: int64


What follows is a key contributution since the Almost There checkpoint.

In [ ]:
# This is my key function. I use 100 epochs for much better performance, a perfectly balanced class distribtuion within the training set (50& control, 50% mass -- not the classweighting
# equation I previously used) implement early stopping for efficiency, and evaluate savling loss and accuracy curves.

def train_model_with_class_weighting(train_metadata, test_metadata, image_folder,
                            train_size,
                            image_size=(224, 224), batch_size=32, epochs=100,
                            output_dir="/content/drive/MyDrive/NIH_ChestXRay_Data_Neuro240/final_results"):
    import os
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, f1_score
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from tensorflow.keras.callbacks import EarlyStopping
    from sklearn.utils.class_weight import compute_class_weight

    os.makedirs(output_dir, exist_ok=True)
    result_prefix = os.path.join(output_dir, f"train_{train_size}")

    # Sample balanced subset
    half_size = train_size // 2
    mass_subset = train_metadata[train_metadata["Finding Labels"] == "Mass"].sample(half_size, random_state=42)
    no_finding_subset = train_metadata[train_metadata["Finding Labels"] == "No Finding"].sample(half_size, random_state=42)
    train_metadata_subset = pd.concat([mass_subset, no_finding_subset]).sample(frac=1, random_state=42)
    test_metadata_subset = test_metadata.sample(int(train_size / 4), random_state=42)

    # Save class distribution
    def save_distribution(df, name):
        counts = df["Finding Labels"].value_counts()
        percents = counts / counts.sum() * 100
        dist_df = pd.DataFrame({"Count": counts, "Percent": percents})
        dist_df.to_csv(f"{result_prefix}_{name}_distribution.csv")
        return dist_df

    save_distribution(train_metadata_subset, "train")
    save_distribution(test_metadata_subset, "test")

    # Data generators
    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False
    )
    test_datagen = ImageDataGenerator(rescale=1.0 / 255)

    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_metadata_subset,
        directory=image_folder,
        x_col="Image Index",
        y_col="Finding Labels",
        target_size=image_size,
        batch_size=batch_size,
        class_mode="binary",
        shuffle=True
    )
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_metadata_subset,
        directory=image_folder,
        x_col="Image Index",
        y_col="Finding Labels",
        target_size=image_size,
        batch_size=batch_size,
        class_mode="binary",
        shuffle=False
    )

    # Class weights
    y_train_labels = train_metadata_subset["Finding Labels"]
    classes = np.unique(y_train_labels)
    class_weights = compute_class_weight("balanced", classes=classes, y=y_train_labels)
    class_weight_dict = dict(zip(classes, class_weights))

    # Old Sequential Model
    model = Sequential([
        Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation="relu"),
        Dropout(0.5),
        Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    history = model.fit(
        train_generator,
        validation_data=test_generator,
        epochs=epochs,
        class_weight=class_weight_dict,
        callbacks=[early_stop],
        verbose=1
    )

    # Save loss & accuracy curves
    pd.DataFrame(history.history).to_csv(f"{result_prefix}_training_history.csv", index=False)
    for metric in ['loss', 'accuracy']:
        plt.figure()
        plt.plot(history.history[metric], label='Train')
        plt.plot(history.history[f'val_{metric}'], label='Validation')
        plt.title(f'{metric.capitalize()} over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel(metric.capitalize())
        plt.legend()
        plt.grid(True)
        plt.savefig(f"{result_prefix}_{metric}_curve.png")
        plt.close()

    # Evaluate
    test_loss, test_acc = model.evaluate(test_generator)
    y_pred_proba = model.predict(test_generator).flatten()
    y_true = test_generator.classes

    thresholds = np.linspace(0.1, 0.9, 9)
    best_f1 = 0
    best_threshold = 0.5
    best_preds = None

    for t in thresholds:
        preds = (y_pred_proba > t).astype(int)
        f1 = f1_score(y_true, preds)
        print(f"Threshold {t:.2f} → F1 Score: {f1:.4f}")
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = t
            best_preds = preds

    print(f"\nBest Threshold: {best_threshold:.2f} → F1 Score: {best_f1:.4f}")
    auc = roc_auc_score(y_true, y_pred_proba)
    cm = confusion_matrix(y_true, best_preds)

    confusion_df = pd.DataFrame(
        cm,
        index=["Mass", "No Finding"],
        columns=["Predicted Mass", "Predicted No Finding"]
    )
    confusion_df.to_csv(f"{result_prefix}_confusion_matrix.csv")

    results_df = pd.DataFrame({
        "Filename": test_generator.filenames,
        "TrueLabel": y_true,
        "PredictedLabel": best_preds,
        "PredictedProb": y_pred_proba
    })
    results_df.to_csv(f"{result_prefix}_predictions.csv", index=False)

    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC = {auc:.2f}")
    plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{result_prefix}_roc_curve.png")
    plt.close()

    with open(f"{result_prefix}_best_threshold.txt", "w") as f:
        f.write(f"Best threshold: {best_threshold:.2f}, F1: {best_f1:.4f}")

    print(f"Test Accuracy (train size={train_size}): {test_acc * 100:.2f}%")
    print(f"AUC: {auc:.4f}")

    return model, history, test_acc, auc, result_prefix

In [ ]:
# Checking results without early stopping
def train_model_with_class_weighting_nostop(train_metadata, test_metadata, image_folder,
                            train_size,
                            image_size=(224, 224), batch_size=32, epochs=100,
                            output_dir="/content/drive/MyDrive/NIH_ChestXRay_Data_Neuro240/final_results/nostop"):
    import os
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, f1_score
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    from sklearn.utils.class_weight import compute_class_weight
    from tensorflow.keras import Input, Model

    os.makedirs(output_dir, exist_ok=True)
    result_prefix = os.path.join(output_dir, f"train_{train_size}")

    # Sample balanced subset
    half_size = train_size // 2
    mass_subset = train_metadata[train_metadata["Finding Labels"] == "Mass"].sample(half_size, random_state=42)
    no_finding_subset = train_metadata[train_metadata["Finding Labels"] == "No Finding"].sample(half_size, random_state=42)
    train_metadata_subset = pd.concat([mass_subset, no_finding_subset]).sample(frac=1, random_state=42)
    test_metadata_subset = test_metadata.sample(int(train_size / 4), random_state=42)

    # Save class distribution
    def save_distribution(df, name):
        counts = df["Finding Labels"].value_counts()
        percents = counts / counts.sum() * 100
        dist_df = pd.DataFrame({"Count": counts, "Percent": percents})
        dist_df.to_csv(f"{result_prefix}_{name}_distribution.csv")
        return dist_df

    save_distribution(train_metadata_subset, "train")
    save_distribution(test_metadata_subset, "test")

    # Data generators
    train_datagen = ImageDataGenerator(
        rescale=1.0 / 255,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False
    )
    test_datagen = ImageDataGenerator(rescale=1.0 / 255)

    train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_metadata_subset,
        directory=image_folder,
        x_col="Image Index",
        y_col="Finding Labels",
        target_size=image_size,
        batch_size=batch_size,
        class_mode="binary",
        shuffle=True
    )
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=test_metadata_subset,
        directory=image_folder,
        x_col="Image Index",
        y_col="Finding Labels",
        target_size=image_size,
        batch_size=batch_size,
        class_mode="binary",
        shuffle=False
    )

    # Class weights
    y_train_labels = train_metadata_subset["Finding Labels"]
    classes = np.unique(y_train_labels)
    class_weights = compute_class_weight("balanced", classes=classes, y=y_train_labels)
    class_weight_dict = dict(zip(classes, class_weights))

    # Model
    # # Old Sequential Model
    # model = Sequential([
    #     Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
    #     MaxPooling2D(pool_size=(2, 2)),
    #     Conv2D(64, (3, 3), activation="relu"),
    #     MaxPooling2D(pool_size=(2, 2)),
    #     Conv2D(128, (3, 3), activation="relu"),
    #     MaxPooling2D(pool_size=(2, 2)),
    #     Flatten(),
    #     Dense(128, activation="relu"),
    #     Dropout(0.5),
    #     Dense(1, activation="sigmoid")
    # ])

    # I will have to try a functional API model, doing this for Grad-CAM

    inputs = Input(shape=(224, 224, 3))
    x = Conv2D(32, (3, 3), activation="relu")(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(128, (3, 3), activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation="sigmoid")(x)


    model = Model(inputs=inputs, outputs=outputs)
    # Rest of code continues as normal
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


    history = model.fit(
        train_generator,
        validation_data=test_generator,
        epochs=epochs,
        class_weight=class_weight_dict,
        verbose=1
    )

    # Save loss & accuracy curves
    pd.DataFrame(history.history).to_csv(f"{result_prefix}_training_history.csv", index=False)
    for metric in ['loss', 'accuracy']:
        plt.figure()
        plt.plot(history.history[metric], label='Train')
        plt.plot(history.history[f'val_{metric}'], label='Validation')
        plt.title(f'{metric.capitalize()} over Epochs')
        plt.xlabel('Epoch')
        plt.ylabel(metric.capitalize())
        plt.legend()
        plt.grid(True)
        plt.savefig(f"{result_prefix}_{metric}_curve.png")
        plt.close()

    # Evaluate
    test_loss, test_acc = model.evaluate(test_generator)
    y_pred_proba = model.predict(test_generator).flatten()
    y_true = test_generator.classes

    thresholds = np.linspace(0.1, 0.9, 9)
    best_f1 = 0
    best_threshold = 0.5
    best_preds = None

    for t in thresholds:
        preds = (y_pred_proba > t).astype(int)
        f1 = f1_score(y_true, preds)
        print(f"Threshold {t:.2f} → F1 Score: {f1:.4f}")
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = t
            best_preds = preds

    print(f"\nBest Threshold: {best_threshold:.2f} → F1 Score: {best_f1:.4f}")
    auc = roc_auc_score(y_true, y_pred_proba)
    cm = confusion_matrix(y_true, best_preds)

    confusion_df = pd.DataFrame(
        cm,
        index=["Mass", "No Finding"],
        columns=["Predicted Mass", "Predicted No Finding"]
    )
    confusion_df.to_csv(f"{result_prefix}_confusion_matrix.csv")

    results_df = pd.DataFrame({
        "Filename": test_generator.filenames,
        "TrueLabel": y_true,
        "PredictedLabel": best_preds,
        "PredictedProb": y_pred_proba
    })
    results_df.to_csv(f"{result_prefix}_predictions.csv", index=False)

    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC = {auc:.2f}")
    plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{result_prefix}_roc_curve.png")
    plt.close()

    with open(f"{result_prefix}_best_threshold.txt", "w") as f:
        f.write(f"Best threshold: {best_threshold:.2f}, F1: {best_f1:.4f}")

    print(f"Test Accuracy (train size={train_size}): {test_acc * 100:.2f}%")
    print(f"AUC: {auc:.4f}")

    return model, history, test_acc, auc, result_prefix

In [ ]:
# Training the maximal model since this is the key result to understand for interpretibility I think
model_1000_class, history_1000_class, acc_1000_class, auc_1000_class, result_prefix_1000 = train_model_with_class_weighting_nostop(
    train_metadata=train_metadata,
    test_metadata=test_metadata,
    image_folder=image_folder,
    train_size=1000
)


Found 1000 validated image filenames belonging to 2 classes.
Found 250 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 683s 21s/step - accuracy: 0.4569 - loss: 0.8497 - val_accuracy: 0.9200 - val_loss: 0.6745
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.4646 - loss: 0.6946 - val_accuracy: 0.1160 - val_loss: 0.6952
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.4942 - loss: 0.6930 - val_accuracy: 0.9200 - val_loss: 0.5940
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.4765 - loss: 0.7018 - val_accuracy: 0.0920 - val_loss: 0.7465
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.5140 - loss: 0.6929 - val_accuracy: 0.1000 - val_loss: 0.7041
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.5006 - loss: 0.6948 - val_accuracy: 0.8000 - val_loss: 0.6629
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.5077 - loss: 0.6952 - val_accuracy: 0.5400 - val_loss: 0.7179
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.5348 - loss: 0.6912 - val_accuracy: 0.5400 -

In [ ]:
# I also want to just test on the baseline model
model_500_class, history_500_class, acc_500_class, auc_500_class, result_prefix_500 = train_model_with_class_weighting_nostop(
    train_metadata=train_metadata,
    test_metadata=test_metadata,
    image_folder=image_folder,
    train_size=500
)

# And on the minimal model
model_250_class, history_250_class, acc_250_class, auc_250_class, result_prefix_250 = train_model_with_class_weighting_nostop(
    train_metadata=train_metadata,
    test_metadata=test_metadata,
    image_folder=image_folder,
    train_size=250
)

In [ ]:
# Now I will perform Grad-CAM Visualization per the encouragement of my TF!
def generate_gradcam_visualizations(model, test_generator, results_df, result_prefix,
                                    image_folder, num_examples=5):
    import tensorflow as tf
    import cv2
    import numpy as np
    import matplotlib.pyplot as plt
    import os
    from tensorflow.keras.preprocessing import image as keras_image
    import numpy as np

    os.makedirs(f"{result_prefix}_gradcam", exist_ok=True)

    # Force-build model by passing dummy input (fix for Sequential)
    dummy_input = tf.zeros((1, 224, 224, 3))
    _ = model(dummy_input)

    # Get last convolutional layer name
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            last_conv_layer_name = layer.name
            break
    else:
        raise ValueError("No Conv2D layer found in model.")

    # Build Grad-CAM model
    grad_model = tf.keras.models.Model(
        inputs=[model.input],
        outputs=[model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Helper to load and preprocess image
    def load_image(img_path, target_size):
        img = keras_image.load_img(img_path, target_size=target_size)
        array = keras_image.img_to_array(img) / 255.0
        return np.expand_dims(array, axis=0), img

    # Getting indices of false positives and false negatives
    fp_idx = results_df[(results_df["TrueLabel"] == 0) & (results_df["PredictedLabel"] == 1)].index
    fn_idx = results_df[(results_df["TrueLabel"] == 1) & (results_df["PredictedLabel"] == 0)].index

    selected_idx = list(fp_idx[:num_examples]) + list(fn_idx[:num_examples])
    selected_type = ['FP'] * min(num_examples, len(fp_idx)) + ['FN'] * min(num_examples, len(fn_idx))

    for i, idx in enumerate(selected_idx):
        filename = results_df.loc[idx, "Filename"]
        img_path = os.path.join(image_folder, filename)
        img_array, original_img = load_image(img_path, target_size=(224, 224))

        with tf.GradientTape() as tape:
            conv_output, predictions = grad_model(img_array)
            loss = predictions[:, 0]

        grads = tape.gradient(loss, conv_output)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        conv_output = conv_output[0]
        heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_output), axis=-1)

        # Normalize and resize
        heatmap = np.maximum(heatmap, 0)
        heatmap /= np.max(heatmap) + 1e-8
        heatmap = cv2.resize(heatmap, (224, 224))
        heatmap = np.uint8(255 * heatmap)
        heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

        superimposed_img = cv2.addWeighted(np.array(original_img).astype(np.uint8), 0.6, heatmap, 0.4, 0)
        output_path = os.path.join(result_prefix + "_gradcam", f"{selected_type[i]}_{filename}")
        cv2.imwrite(output_path, cv2.cvtColor(superimposed_img, cv2.COLOR_RGB2BGR))

    print(f"Grad-CAM saved for {len(selected_idx)} samples in: {result_prefix}_gradcam")

In [ ]:
# Force-build model for Grad-CAM
import tensorflow as tf

# Build model by passing a real-shape dummy image
model_1000_class(tf.zeros((1, 224, 224, 3)))

import numpy as np
# Now call Grad-CAM
results_df = pd.read_csv(f"{result_prefix_1000}_predictions.csv")
generate_gradcam_visualizations(model_1000_class, test_generator, results_df, result_prefix_1000, image_folder)

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(1, 224, 224, 3))
  warnings.warn(msg)


Grad-CAM saved for 8 samples in: /content/drive/MyDrive/NIH_ChestXRay_Data_Neuro240/final_results/nostop/train_1000_gradcam


In [ ]:
# Train the model without early stopping
model_1000_class, history_1000_class, acc_1000_class, auc_1000_class, result_prefix_1000 = train_model_with_class_weighting_nostop(
    train_metadata=train_metadata,
    test_metadata=test_metadata,
    image_folder=image_folder,
    train_size=1000
)

Found 1000 validated image filenames belonging to 2 classes.
Found 250 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.5167 - loss: 1.2094 - val_accuracy: 0.9080 - val_loss: 0.6925
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.4991 - loss: 0.6932 - val_accuracy: 0.0960 - val_loss: 0.7007
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step - accuracy: 0.5246 - loss: 0.6970 - val_accuracy: 0.6880 - val_loss: 0.6918
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.5386 - loss: 0.6940 - val_accuracy: 0.8640 - val_loss: 0.6837
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.5309 - loss: 0.6927 - val_accuracy: 0.6400 - val_loss: 0.6920
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.5138 - loss: 0.6893 - val_accuracy: 0.6160 - val_loss: 0.6972
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.5100 - loss: 0.6928 - val_accuracy: 0.7680 - val_loss: 0.6871
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.5170 - loss: 0.6926 - val_accuracy: 0.6920 - v